<a href="https://colab.research.google.com/github/tanhata/ArtCrit_Blip/blob/main/Art_Critique_System_w_BLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import KaggleHub library
import kagglehub


# Specify dataset name
DATASET_NAME = "ipythonx/wikiart-gangogh-creating-art-gan"

# Download latest version
path = kagglehub.dataset_download(DATASET_NAME)
# Print path to dataset files
print("Path to dataset files:", path)

100%|██████████| 34.6G/34.6G [08:10<00:00, 75.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2


In [ ]:
import os
import zipfile


## Constants
DATA_PATH = path  # Path to downloaded dataset


## Unzip files
zip_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.zip')]
for zip_file in zip_files:
    with zipfile.ZipFile(os.path.join(DATA_PATH, zip_file), 'r') as zip_ref:
        zip_ref.extractall(DATA_PATH)


## Explore files
print("Files in dataset:")
for root, dirs, files in os.walk(DATA_PATH):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
pip install transformers accelerate torch sentencepiece

In [ ]:
pip install safetensors

In [ ]:
from transformers import LlavaForConditionalGeneration, AutoTokenizer
import torch
from PIL import Image

# Confirm LLaVA model loading
model_id = "liuhaotian/llava-v1.5-7b"
model = LlavaForConditionalGeneration.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Simple inference test
def test_llava_inference(model, tokenizer, image_path):
    # Load image
    image = Image.open(image_path)

    # Prepare input
    prompt = "Describe this artwork in detail"
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate critique
    outputs = model.generate(**inputs)
    critique = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return critique

# Try inference
sample_image = image_paths[0]
critique = test_llava_inference(model, tokenizer, sample_image)
print("LLaVA Critique:", critique)

config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/27.1k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.54G [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import list_repo_files

def download_model_files(model_id):
    try:
        # List available files
        files = list_repo_files(model_id)
        print(f"Files in {model_id}:")
        for file in files:
            print(file)

        # Download required files
        required_files = ["preprocessor_config.json", "tokenizer_config.json", "model.ckpt"]
        for file in required_files:
            if file in files:
                print(f"Downloading {file}...")
                # Download file logic here
            else:
                print(f"{file} not found in model repository")

    except Exception as e:
        print(f"Error downloading model files: {e}")

# Download model files
download_model_files("liuhaotian/llava-v1.5-7b")

In [ ]:
import os
import json
from PIL import Image
from transformers import BlipForConditionalGeneration, AutoTokenizer, BlipProcessor
import torch
from torch.utils.data import Dataset, DataLoader
import traceback
import kagglehub

# Specify dataset name
DATASET_NAME = "ipythonx/wikiart-gangogh-creating-art-gan"

# Download latest version
dataset_path = kagglehub.dataset_download(DATASET_NAME)
print("Path to dataset files:", dataset_path)

# Load the BLIP model and tokenizer
model_id = "Salesforce/blip-image-captioning-base"
model = BlipForConditionalGeneration.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = BlipProcessor.from_pretrained(model_id)

# Find image paths and metadata
def find_images_and_metadata(dataset_path):
    image_paths = []
    image_metadata = []
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

    print(f"Searching for images in: {dataset_path}")

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                full_path = os.path.join(root, file)

                # Extract metadata
                try:
                    artist = os.path.basename(os.path.dirname(full_path))
                    style = os.path.basename(os.path.dirname(os.path.dirname(full_path)))
                except:
                    artist = "Unknown"
                    style = "Unknown"

                image_paths.append(full_path)
                image_metadata.append({
                    'path': full_path,
                    'artist': artist,
                    'style': style
                })

    print(f"Total images found: {len(image_paths)}")
    return image_paths, image_metadata

# Load image paths and metadata
image_paths, image_metadata = find_images_and_metadata(dataset_path)

# Define a dataset class for the preprocessed data
class ArtCritiqueDataset(Dataset):
    def __init__(self, image_paths, image_metadata, max_images=10):
        # Limit to first max_images for testing
        self.image_paths = image_paths[:max_images]
        self.image_metadata = image_metadata[:max_images]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        metadata = self.image_metadata[index]

        # Load and preprocess the image using BLIP processor
        image = Image.open(image_path).convert('RGB')

        # Create a prompt based on metadata
        prompt = f"Provide a detailed critique of this {metadata.get('style', 'unknown')} artwork."

        # Prepare inputs for the model using BLIP processor
        inputs = processor(
            text=prompt,
            images=image,
            return_tensors="pt",
            padding=True
        )

        return {
            'inputs': inputs,
            'metadata': metadata,
            'image_path': image_path
        }

# Create dataset and dataloader
dataset = ArtCritiqueDataset(image_paths, image_metadata)
batch_size = 2  # Reduced batch size
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Function to generate art critiques
def generate_art_critiques(model, data_loader, tokenizer):
    art_critiques = []

    # Ensure model is in evaluation mode
    model.eval()

    # Disable gradient computation
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            print(f"Processing batch {batch_idx + 1}")

            try:
                # Unpack batch
                inputs = batch['inputs']
                metadata = batch['metadata']
                image_paths = batch['image_path']

                # Generate critiques
                outputs = model.generate(
                    pixel_values=inputs['pixel_values'],  # Pixel values of images
                    input_ids=inputs['input_ids'],  # Text input ids
                    attention_mask=inputs['attention_mask'],  # Attention mask
                    max_length=50,  # Maximum length of generated text
                    num_beams=4,  # Number of beams for beam search
                    early_stopping=True  # Stop early if possible
                )

                # Decode generated critiques
                for output, meta, img_path in zip(outputs, metadata, image_paths):
                    # Decode the generated output
                    critique = tokenizer.decode(output, skip_special_tokens=True)

                    art_critiques.append({
                        'image_path': img_path,
                        'artist': meta.get('artist', 'Unknown'),
                        'style': meta.get('style', 'Unknown'),
                        'critique': critique
                    })
                    print(f"Generated critique for {img_path}")

            except Exception as e:
                print(f"Error in batch processing: {e}")
                # Print full traceback for detailed error information
                traceback.print_exc()

    return art_critiques

# Generate art critiques
art_critiques = generate_art_critiques(model, data_loader, tokenizer)

# Save critiques
with open('art_critiques.json', 'w') as f:
    json.dump(art_critiques, f, indent=2)

print(f"Generated {len(art_critiques)} art critiques")

In [ ]:
import os
import json
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
import traceback
import kagglehub

# Specify dataset name
DATASET_NAME = "ipythonx/wikiart-gangogh-creating-art-gan"

# Download latest version
dataset_path = kagglehub.dataset_download(DATASET_NAME)
print("Path to dataset files:", dataset_path)

# Load the BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Find image paths and metadata
def find_images_and_metadata(dataset_path):
    image_paths = []
    image_metadata = []
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

    print(f"Searching for images in: {dataset_path}")

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                full_path = os.path.join(root, file)

                # Extract metadata
                try:
                    artist = os.path.basename(os.path.dirname(full_path))
                    style = os.path.basename(os.path.dirname(os.path.dirname(full_path)))
                except:
                    artist = "Unknown"
                    style = "Unknown"

                image_paths.append(full_path)
                image_metadata.append({
                    'path': full_path,
                    'artist': artist,
                    'style': style
                })

    print(f"Total images found: {len(image_paths)}")
    return image_paths, image_metadata

# Load image paths and metadata
image_paths, image_metadata = find_images_and_metadata(dataset_path)

# Function to process a single image
def process_image(image_path, style):
    try:
        # Open the image
        image = Image.open(image_path).convert('RGB')

        # Prepare the prompt
        prompt = f"Provide a detailed critique of this {style} artwork."

        # Process the image and text
        inputs = processor(image, prompt, return_tensors="pt")

        return inputs
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# Function to generate art critiques
def generate_art_critiques(model, processor, image_paths, image_metadata, max_images=10):
    art_critiques = []

    # Ensure model is on the right device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Process a limited number of images
    for idx, (image_path, metadata) in enumerate(zip(image_paths[:max_images], image_metadata[:max_images])):
        print(f"Processing image {idx + 1}/{min(max_images, len(image_paths))}")

        try:
            # Prepare inputs
            inputs = process_image(image_path, metadata.get('style', 'unknown'))

            if inputs is None:
                continue

            # Move inputs to the same device as the model
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Generate critique
            outputs = model.generate(**inputs)

            # Decode the generated text
            critique = processor.decode(outputs[0], skip_special_tokens=True)

            # Store the critique
            art_critiques.append({
                'image_path': image_path,
                'artist': metadata.get('artist', 'Unknown'),
                'style': metadata.get('style', 'Unknown'),
                'critique': critique
            })

            print(f"Generated critique for {image_path}")

        except Exception as e:
            print(f"Error generating critique for {image_path}: {e}")
            traceback.print_exc()

    return art_critiques

# Generate art critiques
art_critiques = generate_art_critiques(model, processor, image_paths, image_metadata)

# Save critiques
with open('art_critiques.json', 'w') as f:
    json.dump(art_critiques, f, indent=2)

print(f"Generated {len(art_critiques)} art critiques")

Path to dataset files: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2
Searching for images in: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2
Total images found: 96013
Processing image 1/10


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/eaa4b5ce85a36b4f328f74c5240a5a89c.jpg
Processing image 2/10
Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/3631418c00783dc2f219951616e892e9c.jpg
Processing image 3/10
Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/8298c3e658217f7e65ed55cca245c52bc.jpg
Processing image 4/10
Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/f1527f3a1004a3a1ef0943fb4aae34b7c.jpg
Processing image 5/10
Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/bd5e8300d8b5db0b4c60e3a83bd689b6c.jpg
Processing image 6/10
Generated critique for /root/.

In [ ]:
# Load and print the art critiques
with open('art_critiques.json', 'r') as f:
    art_critiques = json.load(f)

# Print each critique in a formatted way
for i, critique in enumerate(art_critiques, 1):
    print(f"\n{'='*50}")
    print(f"Critique {i}")
    print(f"{'='*50}")
    print(f"Image Path: {critique['image_path']}")
    print(f"Style: {critique['style']}")
    print(f"Artist: {critique['artist']}")
    print("\nCritique:")
    print(critique['critique'])


Critique 1
Image Path: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/eaa4b5ce85a36b4f328f74c5240a5a89c.jpg
Style: images
Artist: religious-painting

Critique:
provide a detailed critique of this images artwork., c c, c c c c c c

Critique 2
Image Path: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/3631418c00783dc2f219951616e892e9c.jpg
Style: images
Artist: religious-painting

Critique:
provide a detailed critique of this images artwork.s

Critique 3
Image Path: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/8298c3e658217f7e65ed55cca245c52bc.jpg
Style: images
Artist: religious-painting

Critique:
provide a detailed critique of this images artwork.s

Critique 4
Image Path: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/f1527f3a1004a3

In [ ]:
import os
import json
from PIL import Image
from transformers import BlipForConditionalGeneration, AutoTokenizer, BlipProcessor
import torch
from torch.utils.data import Dataset, DataLoader
import traceback
import kagglehub

# Specify dataset name
DATASET_NAME = "ipythonx/wikiart-gangogh-creating-art-gan"

# Download latest version
dataset_path = kagglehub.dataset_download(DATASET_NAME)
print("Path to dataset files:", dataset_path)

# Load the BLIP model and tokenizer
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Find image paths and metadata
def find_images_and_metadata(dataset_path):
    image_paths = []
    image_metadata = []
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

    print(f"Searching for images in: {dataset_path}")

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                full_path = os.path.join(root, file)

                # Extract metadata
                try:
                    artist = os.path.basename(os.path.dirname(full_path))
                    style = os.path.basename(os.path.dirname(os.path.dirname(full_path)))
                except:
                    artist = "Unknown"
                    style = "Unknown"

                image_paths.append(full_path)
                image_metadata.append({
                    'path': full_path,
                    'artist': artist,
                    'style': style
                })

    print(f"Total images found: {len(image_paths)}")
    return image_paths, image_metadata

# Load image paths and metadata
image_paths, image_metadata = find_images_and_metadata(dataset_path)

# Function to process a single image
def process_image(image_path, style):
    try:
        # Open the image
        image = Image.open(image_path).convert('RGB')

        # Prepare the prompt
        prompt = f"Describe the artwork in the style of {style}."

        # Process the image and text
        inputs = processor(image, prompt, return_tensors="pt")

        return inputs
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# Function to generate art critiques
def generate_art_critiques(model, processor, image_paths, image_metadata, max_images=10):
    art_critiques = []

    # Ensure model is on the right device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Process a limited number of images
    for idx, (image_path, metadata) in enumerate(zip(image_paths[:max_images], image_metadata[:max_images])):
        print(f"Processing image {idx + 1}/{min(max_images, len(image_paths))}")

        try:
            # Prepare inputs
            inputs = process_image(image_path, metadata.get('style', 'unknown'))

            if inputs is None:
                continue

            # Move inputs to the same device as the model
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Generate critique
            outputs = model.generate(**inputs)

            # Decode the generated text
            critique = processor.decode(outputs[0], skip_special_tokens=True)

            # Store the critique
            art_critiques.append({
                'image_path': image_path,
                'artist': metadata.get('artist', 'Unknown'),
                'style': metadata.get('style', 'Unknown'),
                'critique': critique
            })

            print(f"Generated critique for {image_path}")

        except Exception as e:
            print(f"Error generating critique for {image_path}: {e}")
            traceback.print_exc()

    return art_critiques

# Generate art critiques
art_critiques = generate_art_critiques(model, processor, image_paths, image_metadata)

# Save critiques
with open('art_critiques.json', 'w') as f:
    json.dump(art_critiques, f, indent=2)

print(f"Generated {len(art_critiques)} art critiques")

Path to dataset files: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2
Searching for images in: /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2
Total images found: 96013
Processing image 1/10
Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/eaa4b5ce85a36b4f328f74c5240a5a89c.jpg
Processing image 2/10
Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/3631418c00783dc2f219951616e892e9c.jpg
Processing image 3/10
Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/8298c3e658217f7e65ed55cca245c52bc.jpg
Processing image 4/10
Generated critique for /root/.cache/kagglehub/datasets/ipythonx/wikiart-gangogh-creating-art-gan/versions/2/images/religious-painting/f1527f3a1004a3a1ef